In [3]:
import db_utils
import yaml, pandas as pd, numpy as np

with open('credentials.yaml') as file:
    credentials_dict = yaml.safe_load(file)
    
credentials = db_utils.RDSDatabaseConnector(credentials_dict)
loan_payments = credentials.initialise_database()

credentials.save_file(loan_payments)
database = pd.read_csv('new_file.csv', index_col = 'id')

cleaned_data = db_utils.DataTransform(database)

date_data = ['issue_date', 'earliest_credit_line', 'last_payment_date', 'next_payment_date',
             'last_credit_pull_date']

categorical_data = ['member_id', 'term', 'int_rate', 'grade', 'sub_grade', 'employment_length', 'home_ownership', 'verification_status', 'loan_status', 
                    'payment_plan', 'purpose', 'policy_code', 'application_type']

non_numeric_data = date_data + categorical_data 
column_headings = database.columns.values.tolist()
numeric_data = [column for column in column_headings if column not in non_numeric_data]

for date_column in date_data:
    database, date_column = cleaned_data.date_data(database, date_column)

for categories in categorical_data:
    database, categories = cleaned_data.change_type(database, categories, 'category')

df_info = db_utils.DataFrameInfo(database)

for column in column_headings:
    null_vals, null_percentage = df_info.missing(database, column)
    null_percentage = round(null_percentage, 2)
    if null_percentage > float(50):
        database = database.drop(column, axis=1)
    elif float(0) < null_percentage < float(50):
        print(f'{column}: {null_vals} null values, {null_percentage}%')

funded_amount_column = list(database['funded_amount'])
loan_amount_column = list(database['loan_amount'])

for index in range(54231):
    if np.isnan(funded_amount_column[index]):
        funded_amount_column[index] = loan_amount_column[index]
database['funded_amount'] = funded_amount_column


funded_amount: 3007 null values, 5.54%
term: 4772 null values, 8.8%
int_rate: 5169 null values, 9.53%
employment_length: 2118 null values, 3.91%
last_payment_date: 73 null values, 0.13%
last_credit_pull_date: 7 null values, 0.01%
collections_12_mths_ex_med: 51 null values, 0.09%
         member_id  loan_amount  funded_amount  funded_amount_inv       term  \
id                                                                             
38676116  41461848         8000         8000.0             8000.0  36 months   
38656203  41440010        13200        13200.0            13200.0  36 months   
38656154  41439961        16000        16000.0            16000.0  36 months   
38656128  41439934        15000        15000.0            15000.0  36 months   
38656121  41439927        15000        15000.0            15000.0  36 months   
38656111  41439917         2525         2525.0             2525.0  36 months   
38656110  41439916         6675         6675.0             6675.0        NaN   
